<a href="https://colab.research.google.com/github/sreent/large-language-model/blob/main/3C%20Multi-Stage%20Reasoning%20with%20HuggingFace's%20Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HuggingFace's Agent

From Transformers version v4.29 introduces a new API: an API of **tools** and **agents** 🤩

It provides a natural language API on top of transformers: we define a set of curated tools, and design an agent to interpret natural language and to use these tools. It is extensible by design; we curated some relevant tools, but we'll show you how the system can be extended easily to use any tool.

Let's start with a few examples of what can be achieved with this new API. It is particularly powerful when it comes to multimodal tasks, so let's take it for a spin to generate images and read text out loud.

The accompanying docs are [Transformers Agent](https://huggingface.co/docs/transformers/en/transformers_agents) and [Custom Tools](https://huggingface.co/docs/transformers/en/custom_tools).

In [1]:
!pip -q install transformers[torch]==4.32.0
!pip -q install huggingface-hub==0.16.4
!pip -q install datasets==2.14.4
!pip -q install openai==0.27.9
!pip -q install diffusers==0.20.0 accelerate==0.21.0 soundfile==0.12.1
!pip -q install sentencepiece==0.1.99 opencv-python==4.8.0.76

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import IPython
import soundfile as sf
import numpy

def play_audio(audio):
    sf.write("speech_converted.wav", audio.numpy(), samplerate=16000)
    return IPython.display.Audio("speech_converted.wav")

## Generate API tokens
For many of the services that we'll using in the notebook, we'll need some API keys. Follow the instructions below to generate your own.

### Hugging Face Hub
1. Go to this [Inference API page](https://huggingface.co/inference-api) and click "Sign Up" on the top right.

<img src="https://files.training.databricks.com/images/llm/hf_sign_up.png" width=700>

2. Once you have signed up and confirmed your email address, click on your user icon on the top right and click the `Settings` button.

3. Navigate to the `Access Token` tab and copy your token.

<img src="https://files.training.databricks.com/images/llm/hf_token_page.png" width=500>

### OPTIONAL: Use OpenAI's language model

If you'd rather use OpenAI, you need to generate an OpenAI key.

Steps:
1. You need to [create an account](https://platform.openai.com/signup) on OpenAI.
2. Generate an OpenAI [API key here](https://platform.openai.com/account/api-keys).

Note: OpenAI does not have a free option, but it gives you \$5 as credit. Once you have exhausted your \$5 credit, you will need to add your payment method. You will be [charged per token usage](https://openai.com/pricing).

**IMPORTANT**: It's crucial that you keep your OpenAI API key to yourself. If others have access to your OpenAI key, they will be able to charge their usage to your account!


### Create Environment File
To use LLM models and external services, we need to add access token for HuggingFace and API keys for SerpApi and OpenAI to our environment path.

1. Create <code>secret.json</code> and place it in our GDrive.
2. Add entries, i.e. key-value pairs in the following format:
```{code-block}
{
    "HUGGINGFACEHUB_API_TOKEN": "<FILL IN>",
    "OPENAI_API_KEY": "<FILL IN>"
}
```



In [ ]:
import os, json
from google.colab import drive

drive.mount("/content/drive")

# path to api keys, i.e. where secrets.json is stored in GDrive
SECRET_FILE_PATH = "/content/drive/MyDrive/secrets.json"

# load environment variables, i.e. tokens or api keys required for HuggingFace, SerpApi and OpenAI API usages
with open(SECRET_FILE_PATH, "r") as f :
    secrets = json.loads(f.read())

    for (key, value) in secrets.items() :
        os.environ[key] = value

# Do anything with Transformers

We'll start by instantiating an **agent**, which is a large language model (LLM).

We recommend using the OpenAI for the best results, but fully open-source models such as StarCoder or OpenAssistant are also available.

## Agent Initialization

In [ ]:
from transformers.tools import OpenAiAgent

agent = OpenAiAgent(model="text-davinci-003", api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
from transformers.tools import HfAgent

#agent = HfAgent("https://api-inference.huggingface.co/models/bigcode/starcoder")

## Using the agent

The agent is initialized! We now have access to the full power of the tools it has access to.

Let's use it 😎

In [ ]:
boat = agent.run("Generate an image of a boat in the water")
boat

If you'd like to hand objects (or previous results!) to the agent, you can do so by passing a variable directly, and mentioning between backticks the name of the variable passed. For example, if I want to re-use the previous boat generation:

In [ ]:
caption = agent.run("Can you caption the `boat_image`?", boat_image=boat)
caption

Agents vary in competency and their capacity to handle several instructions at once; however the strongest of them (such as OpenAI's) are able to handle complex instructions such as the following three-part instruction:

In [ ]:
audio = agent.run("Can you generate an image of a boat? Please read out loud the contents of the image afterwards")
play_audio(audio)

Where this works great is when your query implies the use of tools which you haven't described directly. An exemple of this would be the following query: "Read out loud the summary of hf.co"

Here we're asking the model to perform three steps at once:
- Fetch the website https://huggingface.co
- Summarize it
- Translate the text to speech

In [ ]:
audio = agent.run("Read out loud the summary of http://hf.co")
play_audio(audio)

Using the best agents works well 🎉

### Chat mode

So far, we've been using the agent by using it's `.run` command. But that's not the only command it has access to; the second command it has access to is `.chat`, which enables using it in chat mode.

The difference between the two is relative to their memory:
- `.run` does not keep memory across runs, but performs better for multiple operations at once (such as running two, or three tools in a row from a given instruction)
- `.chat` keeps memory across runs, but performs better at single instructions.

Let's use it in chat mode!

In [ ]:
agent.chat("Show me an an image of a capybara")

What if we wanted to change something in the image? For example, move the capybaras to a snowy environment

In [ ]:
agent.chat("Transform the image so that it snows")

Now what if we wanted to remove the capybara in favor of something else? We could ask it to show us a mask of the capybara in the image:

In [ ]:
agent.chat("Show me a mask of the snowy capybaras")

Having access to the past history is great to repeatedly iterate on a given prompt. However, it has its limitations and sometimes you'd like to have a clean history. In order to do so, you can use the following method:

In [ ]:
agent.prepare_for_new_chat()

## Tools

So far we've been using the tools that the agent has access to. These tools are the following:

- **Document question answering**: given a document (such as a PDF) in image format, answer a question on this document (Donut)
- **Text question answering**: given a long text and a question, answer the question in the text (Flan-T5)
- **Unconditional image captioning**: Caption the image! (BLIP)
- **Image question answering**: given an image, answer a question on this image (VILT)
- **Image segmentation**: given an image and a prompt, output the segmentation mask of that prompt (CLIPSeg)
- **Speech to text**: given an audio recording of a person talking, transcribe the speech into text (Whisper)
- **Text to speech**: convert text to speech (SpeechT5)
- **Zero-shot text classification**: given a text and a list of labels, identify to which label the text corresponds the most (BART)
- **Text summarization**: summarize a long text in one or a few sentences (BART)
- **Translation**: translate the text into a given language (NLLB)

We also support the following community-based tools:

- **Text downloader**: to download a text from a web URL
- **Text to image**: generate an image according to a prompt, leveraging stable diffusion
- **Image transformation**: transforms an image

We can therefore use a mix and match of different tools by explaining in natural language what we would like to do.

But what about adding new tools? Let's take a look at how to do that

### Adding new tools

We'll add a very simple tool so that the demo remains simple: we'll use the awesome cataas (Cat-As-A-Service) API to get random cats on each run.

We can get a random cat with the following code:

In [ ]:
import requests
from PIL import Image

image = Image.open(requests.get('https://cataas.com/cat', stream=True).raw)
image

Let's create a tool that can be used by our system!

All tools depend on the superclass Tool that holds the main attributes necessary. We'll create a class that inherits from it:

In [ ]:
from transformers import Tool

class CatImageFetcher(Tool):
    pass

This class has a few needs:

- An attribute name, which corresponds to the name of the tool itself. To be in tune with other tools which have a performative name, we'll name it text-download-counter.
- An attribute description, which will be used to populate the prompt of the agent.
- inputs and outputs attributes. Defining this will help the python interpreter make educated choices about types, and will allow for a gradio-demo to be spawned when we push our tool to the Hub. They're both a list of expected values, which can be text, image, or audio.
- A __call__ method which contains the inference code. This is the code we've played with above!

Here’s what our class looks like now:

In [ ]:
from transformers import Tool
from huggingface_hub import list_models


class CatImageFetcher(Tool):
    name = "cat_fetcher"
    description = ("This is a tool that fetches an actual image of a cat online. It takes no input, and returns the image of a cat.")

    inputs = []
    outputs = ["text"]

    def __call__(self):
        return Image.open(requests.get('https://cataas.com/cat', stream=True).raw).resize((256, 256))

We can simply use and test the tool directly:

In [ ]:
tool = CatImageFetcher()
tool()

In order to pass the tool to the agent, we recommend instantiating the agent with the tools directly:

In [ ]:
from transformers.tools import HfAgent

agent = HfAgent("https://api-inference.huggingface.co/models/bigcode/starcoder", additional_tools=[tool])

Let's try to have the agent use it with other tools!

In [ ]:
agent.run("Fetch an image of a cat online and caption it for me")

Success 🎉

The tool was used to fetch a cat image, and the image captioning tool was used shortly after in order to caption that same image.

Finally, we recommend pushing the tool to the Hub in order to have others benefit from it. Here is the documentation that contains more information in order to do so: [Adding a new tool](https://huggingface.co/docs/transformers/en/custom_tools#creating-a-new-tool)

Thanks for following through with the notebook! We're looking forward to the tools you'll push, which will help empower all agents.

In [ ]:
agent.run(
    "Create a dataset (DO NOT try to download one, you MUST create one based on what you find) on the performance of the Mercedes AMG F1 team in 2020 and do some analysis. You need to plot your results. The data can be found in https://en.wikipedia.org/wiki/Mercedes-Benz_in_Formula_One"
)